In [3]:
import tensorflow as tf 
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import EfficientNetB0
from keras.layers import GlobalAveragePooling2D, Dense
from keras.layers import BatchNormalization, Dropout
from keras.models import Model

In [4]:
from tensorflow.keras.datasets import cifar10

In [5]:
# setting class names
class_names=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
#loading the dataset
(x_train,y_train),(x_test,y_test)=cifar10.load_data()


In [6]:
# Normalizing Images

x_train=x_train/255.0
x_train.shape
x_test=x_test/255.0
x_test.shape

(10000, 32, 32, 3)

ResNet50

In [7]:
res = ResNet50(weights ='imagenet', include_top = False, 
               input_shape =(32,32, 3)) 
res.trainable = False

In [14]:
x= res.output

x = GlobalAveragePooling2D()(x)

x = BatchNormalization()(x)

x = Dropout(0.5)(x) 
x = Dense(512, activation ='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)


x = Dense(10, activation ='softmax')(x)

model = Model(res.input, x)
model.compile(optimizer ='Adam', 
              loss ="sparse_categorical_crossentropy", 
              metrics =["sparse_categorical_accuracy"]) 
model.summary() 
model.fit(x_train,y_train, epochs = 5, validation_data = (x_test,y_test))


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________________

NameError: ignored

In [15]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print("Test accuracy: {}".format(test_accuracy))

313/313 [==============================] - 37s 117ms/step - loss: 1.6508 - sparse_categorical_accuracy: 0.4178
Test accuracy: 0.41780000925064087


VGG16

In [16]:
vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=(32,32, 3))
vgg16.trainable = False

In [18]:
x = vgg16.output

x = GlobalAveragePooling2D()(x)

x = BatchNormalization()(x)

x = Dropout(0.5)(x) 
x = Dense(512, activation ='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)


x = Dense(10, activation ='softmax')(x)

model = Model(vgg16.input, x)
model.compile(optimizer ='Adam', 
              loss ="sparse_categorical_crossentropy", 
              metrics =["sparse_categorical_accuracy"]) 
model.summary() 
model.fit(x_train,y_train, epochs = 5, validation_data = (x_test,y_test))
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print("Test accuracy: {}".format(test_accuracy))

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

 EfficientNet


In [21]:
enet = EfficientNetB0(weights ='imagenet', include_top = False, 
               input_shape =(32,32, 3)) 
enet.trainable = False

In [22]:
x = enet.output

x = GlobalAveragePooling2D()(x)

x = BatchNormalization()(x)

x = Dropout(0.5)(x) 
x = Dense(512, activation ='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)


x = Dense(10, activation ='softmax')(x)

model = Model(enet.input, x)
model.compile(optimizer ='Adam', 
              loss ="sparse_categorical_crossentropy", 
              metrics =["sparse_categorical_accuracy"]) 
model.summary() 
model.fit(x_train,y_train, epochs = 5, validation_data = (x_test,y_test))
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print("Test accuracy: {}".format(test_accuracy))

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
rescaling_1 (Rescaling)         (None, 32, 32, 3)    0           input_4[0][0]                    
__________________________________________________________________________________________________
normalization_1 (Normalization) (None, 32, 32, 3)    7           rescaling_1[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 33, 33, 3)    0           normalization_1[0][0]            
____________________________________________________________________________________________

I Analysed 3 Transfer Learning Cases with ResNet50, VGG16 and EfficientNetB0. 
One of the major Reasons why all 3 didnt gave a decent accuracy I think is the Image Size. Cifar-10 Has 32x32 and ResNet had by default 256x256. Issue with making it 32x32 for our purpose is, the tensor size keeps on decreasing as convolutional layers progress and reaches sizes like (1, 1, 512) and continues for more than 3 layers and it is evident that 1x1 tensor cannot lern any significant pattern and it affects the overall performance.

What I Understood is that Image size and model we choose should be compatible enough to learn well and layers we add above also matter a lot.

Test Accuracies on Cifar-10 dataset

ResNet50: 41.7&

VGG16:56.1%


EfficientNetB0:10%
